### API Yahoo Finance

In [1]:
# previamente se  instala las siguientes librerias "yfinance".

In [2]:
# librería para descargar con yahoo finance
import yfinance as yf
# librería para manipular dataframes
import pandas as pd
# librería para manipular fechas
import datetime as dt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Diccionario con las empresas de las que se traen los datos
dict_Empresas = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Meta": "META",
    "Tesla": "TSLA",

    # Algunas sugerencias:
    
    #"Alphabet": "GOOGL",
    #"Amazon": "AMZN",
    #"JPMorgan": "JPM",
    #"Johnson & Johnson": "JNJ",
    #"Walmart": "WMT"
    }

In [4]:
# Mediante un bucle, se toman los datos de cada empresa y se guardan en la carpeta Datasets
# Cada key es el nombre de la empresa, y value es la abreviación para tomar los datos desde finance
for key, value in dict_Empresas.items():
    # Se descargan los datos desde 01/01/23 hasta hoy
    Dataset = yf.download(value, start = dt.date(2023,1,1), end = dt.date.today())

    # Se guardan en un archivo con el nombre de la empresa
    Dataset.to_csv(f"Datasets/{key}.csv")

    # Se imprime en pantalla un mensaje de confirmacion
    print(f"Archivo {key} cargado exitosamente.")

[*********************100%***********************]  1 of 1 completed
Archivo Apple cargado exitosamente.
[*********************100%***********************]  1 of 1 completed
Archivo Microsoft cargado exitosamente.
[*********************100%***********************]  1 of 1 completed
Archivo Meta cargado exitosamente.
[*********************100%***********************]  1 of 1 completed
Archivo Tesla cargado exitosamente.


### SQL

In [5]:
import sqlite3

# Conexión a la base de datos
conn = sqlite3.connect('./Database/test.db')
conn.close()


In [6]:
Dataset
#Que es?

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,118.470001,118.800003,104.639999,108.099998,108.099998,231402800
2023-01-04,109.110001,114.589996,107.519997,113.639999,113.639999,180389000
2023-01-05,110.510002,111.750000,107.160004,110.339996,110.339996,157986300
2023-01-06,103.000000,114.389999,101.809998,113.059998,113.059998,220911100
2023-01-09,118.959999,123.519997,117.110001,119.769997,119.769997,190284000
...,...,...,...,...,...,...
2023-04-27,152.639999,160.479996,152.369995,160.190002,160.190002,127015200
2023-04-28,160.899994,165.000000,157.320007,164.309998,164.309998,122515800
2023-05-01,163.169998,163.279999,158.830002,161.830002,161.830002,109015000


In [9]:
import os

# Carpeta con los archivos CSV
folder_path = './Datasets'

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

files

['Apple.csv', 'Meta.csv', 'Microsoft.csv', 'Tesla.csv']

In [12]:
# Lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# Leer cada archivo CSV en un DataFrame y agregarlo a la lista
for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df['Empresa'] = file.split('.')[0]
        dfs.append(df)

# Concatenar los DataFrames en uno solo
result = pd.concat(dfs, ignore_index=True)

# Imprimir el resultado
result.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Empresa
0,2023-01-03,130.279999,130.899994,124.169998,125.070000,124.879326,112117500,Apple
1,2023-01-04,126.889999,128.660004,125.080002,126.360001,126.167366,89113600,Apple
2,2023-01-05,127.129997,127.769997,124.760002,125.019997,124.829399,80962700,Apple
3,2023-01-06,126.010002,130.289993,124.889999,129.619995,129.422394,87754700,Apple
4,2023-01-09,130.470001,133.410004,129.889999,130.149994,129.951584,70790800,Apple


In [14]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
conn.close()

Connected to SQLite


In [18]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='append')
conn.close()

Connected to SQLite


In [19]:
DB = './Database/stock_db.db'
conn = sqlite3.connect(DB)
print("Connected to SQLite")
result.to_sql('main', conn, schema=None, 
              if_exists='replace')

Connected to SQLite


336

In [20]:
# Creo una función que haga todo:

def write_sql_main(DB, df, table):
	"""Esta funcion inserta masivamente un df o una lista de tuplas en SQL."""
	try:
		conn = sqlite3.connect(DB)
		print("Connected to SQLite")
		df.to_sql(table, conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)
		result = 'ok'
	except sqlite3.Error as error:
		print("Failed to insert multiple records into sqlite table", error)
		result = 'failed'
	finally:
		if conn:
			conn.close()
			print("The SQLite connection is closed")
	return result

In [21]:
write_sql_main(DB=DB, df = result, table = 'main2')

Connected to SQLite
The SQLite connection is closed


'ok'